In [2]:
import fasttext
from tira.rest_api_client import Client

In [3]:
tira = Client()

text = tira.pd.inputs(
    "nlpbuw-fsu-sose-24", "language-identification-train-20240429-training"
)
text = text.set_index("id")
labels = tira.pd.truths(
    "nlpbuw-fsu-sose-24", "language-identification-train-20240429-training"
)
df_train = text.join(labels.set_index("id"))

with open("train_data.txt", "w", encoding="utf-8") as f:
    for idx, row in df_train.iterrows():
        line = row['text'] + " __label__" + row['lang'] + "\n"
        f.write(line)

In [4]:
model = fasttext.train_supervised(input="train_data.txt", epoch=10, lr=0.1)

Read 23M words
Number of words:  3353931
Number of labels: 20
Progress: 100.0% words/sec/thread: 2459300 lr:  0.000000 avg.loss:  0.027941 ETA:   0h 0m 0s


In [13]:
text = tira.pd.inputs(
    "nlpbuw-fsu-sose-24", "language-identification-validation-20240429-training"
)
text = text.set_index("id")
labels = tira.pd.truths(
    "nlpbuw-fsu-sose-24", "language-identification-validation-20240429-training"
)
df = text.join(labels.set_index("id"))
predictions = [model.predict(text)[0][0] for text in df["text"]]

: 

In [9]:
def convert_labels_to_lang(predictions):
    lang_labels = {
        '__label__af': 'af',
        '__label__az': 'az',
        '__label__bg': 'bg',
        '__label__cs': 'cs',
        '__label__da': 'da',
        '__label__de': 'de',
        '__label__el': 'el',
        '__label__en': 'en',
        '__label__es': 'es',
        '__label__fi': 'fi',
        '__label__fr': 'fr',
        '__label__hr': 'hr',
        '__label__it': 'it',
        '__label__ko': 'ko',
        '__label__nl': 'nl',
        '__label__no': 'no',
        '__label__pl': 'pl',
        '__label__ru': 'ru',
        '__label__ur': 'ur',
        '__label__zh': 'zh'
    }
    converted_predictions = []
    for prediction in predictions:
        if prediction in lang_labels:
            converted_predictions.append(lang_labels[prediction])

    return converted_predictions

In [10]:
preds= convert_labels_to_lang(predictions)

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_classification(true_labels, predicted_labels):
    """
    Berechnet die Accuracy, Precision, Recall und F1-Score für eine Klassifikationsaufgabe.

    Args:
    true_labels (list): Die echten Labels.
    predicted_labels (list): Die vorhergesagten Labels.

    Returns:
    dict: Ein Dictionary mit Accuracy, Precision, Recall und F1-Score.
    """
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='macro')
    recall = recall_score(true_labels, predicted_labels, average='macro')
    f1 = f1_score(true_labels, predicted_labels, average='macro')

    evaluation_results = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }

    return evaluation_results

# Beispielaufruf:
evaluation_results = evaluate_classification(df["lang"], preds)
print("Evaluationsergebnisse:", evaluation_results)

Evaluationsergebnisse: {'Accuracy': 0.997125, 'Precision': 0.9971408205889292, 'Recall': 0.9971250000000002, 'F1-Score': 0.9971291066831128}
